In [6]:
import numpy as np
import cv2
import h5py
import tensorflow as tf
import nibabel as nib
import nibabel as nib
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
import cv2
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [7]:
# dice loss as defined above for 4 classes
def dice_coef(y_true, y_pred, smooth=1.0):
    class_num = 4
    for i in range(class_num):
        y_true_f = K.flatten(y_true[:,:,:,i])
        y_pred_f = K.flatten(y_pred[:,:,:,i])
        intersection = K.sum(y_true_f * y_pred_f)
        loss = ((2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))
   #     K.print_tensor(loss, message='loss value for class {} : '.format(SEGMENT_CLASSES[i]))
        if i == 0:
            total_loss = loss
        else:
            total_loss = total_loss + loss
            
    total_loss = total_loss / class_num
#    K.print_tensor(total_loss, message=' total dice coef: ')
    return total_loss

def dice_coef_necrotic(y_true, y_pred, epsilon=1e-6):
    intersection = K.sum(K.abs(y_true[:,:,:,1] * y_pred[:,:,:,1]))
    return (2. * intersection) / (K.sum(K.square(y_true[:,:,:,1])) + K.sum(K.square(y_pred[:,:,:,1])) + epsilon)

def dice_coef_edema(y_true, y_pred, epsilon=1e-6):
    intersection = K.sum(K.abs(y_true[:,:,:,2] * y_pred[:,:,:,2]))
    return (2. * intersection) / (K.sum(K.square(y_true[:,:,:,2])) + K.sum(K.square(y_pred[:,:,:,2])) + epsilon)

def dice_coef_enhancing(y_true, y_pred, epsilon=1e-6):
    intersection = K.sum(K.abs(y_true[:,:,:,3] * y_pred[:,:,:,3]))
    return (2. * intersection) / (K.sum(K.square(y_true[:,:,:,3])) + K.sum(K.square(y_pred[:,:,:,3])) + epsilon)



# Computing Precision 
def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    
# Computing Sensitivity      
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())


# Computing Specificity
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [8]:
from tensorflow.keras.models import load_model

model = load_model("/home/amit/DSML_Brats/model_2024_2D_UNet.h5", custom_objects= {'dice_coef': dice_coef, 'precision': precision, 'sensitivity':sensitivity, 'specificity': specificity, 'dice_coef_necrotic':dice_coef_necrotic,'dice_coef_edema':dice_coef_edema,'dice_coef_enhancing':dice_coef_enhancing })

In [ ]:
data_path = "/home/amit/DSML_Brats/Dataset/"
image_file = nib.load(os.path.join(data_path, "BraTS-GLI-00006-101-t2f.nii.gz"))
image_file2 = nib.load(os.path.join(data_path, "BraTS-GLI-00006-101-t1c.nii.gz"))
# label_file = nib.load(os.path.join(data_path, "BraTS-GLI-00006-101-seg.nii.gz"))

image = image_file.get_fdata()  
image2 = image_file2.get_fdata()  
# label = label_file.get_fdata()  

# Normalize image
image = (image - np.min(image)) / (np.max(image) - np.min(image))
image2 = (image2 - np.min(image2)) / (np.max(image2) - np.min(image2))

In [12]:
# Load and process the images
slice_idx = image.shape[2] // 2  # Middle slice

# Resize the images
IMG_SIZE = 128
flair_resized = cv2.resize(image[:, :, slice_idx], (IMG_SIZE, IMG_SIZE))
ce_resized = cv2.resize(image2[:, :, slice_idx], (IMG_SIZE, IMG_SIZE))

# Stack them together to create the 2-channel input
X_input = np.stack((flair_resized, ce_resized), axis=-1)  # Shape will be (IMG_SIZE, IMG_SIZE, 2)

# Normalize the input
X_input = X_input / np.max(X_input)  # Normalizing the input to [0, 1]

# If you need to add an extra batch dimension
X_input = np.expand_dims(X_input, axis=0)  # Shape will be (1, IMG_SIZE, IMG_SIZE, 2)

In [13]:
# If you also want to process the label
label_resized = cv2.resize(label[:, :, slice_idx], (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
label_resized[label_resized >= 4] = 3  # Map to the last class
label_one_hot = tf.one_hot(label_resized, 4)  # Convert to one-hot encoding (4 classes)

In [14]:
# Expand dimensions for batch size (if necessary)
label_one_hot = np.expand_dims(label_one_hot, axis=0)  # Shape will be (1, IMG_SIZE, IMG_SIZE, 4)

In [15]:
# Make a prediction
prediction = model.predict(X_input)

2024-11-25 17:39:21.017818: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-11-25 17:39:21.128291: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-11-25 17:39:21.421689: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 548.16MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-11-25 17:39:21.456456: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 548.16MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-11-25 17:39:21.613308: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) r

1/1 [==============================] - 2s 2s/step


In [ ]:
# Optionally, display the results
import matplotlib.pyplot as plt

# Plot the images
plt.figure(figsize=(10, 6))

# Plot the T2-FLAIR image
plt.subplot(1, 3, 1)
plt.imshow(flair_resized, cmap='gray')
plt.title("T2-FLAIR Slice")
plt.axis("off")

# Plot the T1-contrast image
plt.subplot(1, 3, 2)
plt.imshow(ce_resized, cmap='gray')
plt.title("T1-contrast Slice")
plt.axis("off")

# Plot the prediction (you can choose the channel with the max probability, for example)
plt.subplot(1, 3, 3)
plt.imshow(np.argmax(prediction[0], axis=-1), cmap='jet')  # Take the first batch and argmax over the classes
plt.title("Model Prediction")
plt.axis("off")

plt.show()